# Triple Coincidence Analysis

In [1]:
#Include modules
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from multiprocessing import Pool

In [2]:
# Read the files

# Parameters
run         = 'run0'
header      = ['EventID','TrackID','Particle','EnergyDeposited','XPosition','YPosition','ZPosition','LocalTime','Volume']
PROCESSES   = 7

# Find the files
foldername  = f'./data/{run}/output'
filenames   = os.listdir(foldername)

print(f'Found {len(filenames)} data files in {foldername}') 

# Function to load a single file
def loadfile(filename):
    return pd.read_csv(os.path.join(foldername,filename),skiprows=14,names=header)

# Read the files into a list
def get_files(filenames):

    # Parallelize loading them
    with Pool(PROCESSES) as pool:
        files = list(tqdm(pool.imap(loadfile, filenames),total=len(filenames)))
    return files

files = get_files(filenames)


Found 1024 data files in ./data/run0/output


  0%|          | 0/1024 [00:00<?, ?it/s]

In [3]:
data = files[0]

In [32]:
counts      = data.EventID.value_counts()
filtered    = data[data.EventID.isin(counts.index[counts.gt(2)])]

In [76]:
filtered

,EventID,TrackID,Particle,EnergyDeposited,XPosition,YPosition,ZPosition,LocalTime,Volume
2,88,1,mu-,1.712150,92.21500,544.416,-54.8510,0.354137,physPaddleTop
3,88,1,mu-,1.281910,97.83940,540.000,-59.3213,0.382273,physPaddleTop
4,88,2,e-,0.239067,92.32050,543.806,-54.7890,0.002405,physPaddleTop
5,88,2,e-,0.217733,92.32260,543.673,-54.5337,0.003751,physPaddleTop
12,192,1,mu+,1.125520,-68.90860,543.707,228.9780,0.202060,physPaddleTop
...,...,...,...,...,...,...,...,...,...
58811,488017,2,e-,0.218103,-84.31980,545.644,223.7960,0.005307,physPaddleTop
58816,488127,1,mu-,2.935570,-9.05454,541.122,-28.5874,0.315167,physPaddleTop
58817,488127,1,mu-,0.595469,-7.71346,540.000,-28.1592,0.321170,physPaddleTop
58818,488127,4,e-,0.142576,-9.01703,540.224,-28.7468,0.003109,physPaddleTop


In [72]:

group = filtered.groupby(['EventID','TrackID']).size().reset_index().rename(columns={0:'count'})
group = group[group['count'].apply(lambda x: True if x>=3 else False)]

group 

,EventID,TrackID,count
5,200,14,6
7,375,2,7
11,504,4,5
12,513,1,3
13,513,14,4
...,...,...,...
15193,487180,2,5
15201,487482,20,6
15208,487599,11,15
15218,487811,17,5


In [83]:
def filter(x):
    return 0

filtered.groupby(['EventID','TrackID','Volume'])['Volume']

,0,1
0,"(88, 1, physPaddleTop)",2 physPaddleTop 3 physPaddleTop Name: Vo...
1,"(88, 2, physPaddleTop)",4 physPaddleTop 5 physPaddleTop Name: Vo...
2,"(192, 1, physPaddleTop)",12 physPaddleTop 13 physPaddleTop Name: ...
3,"(192, 2, physPaddleTop)",14 physPaddleTop 15 physPaddleTop Name: ...
4,"(200, 1, physPaddleBottom)",16 physPaddleBottom 17 physPaddleBottom ...
...,...,...
15634,"(487993, 14, physPaddleBottom)",58802 physPaddleBottom 58803 physPaddleB...
15635,"(488017, 1, physPaddleTop)",58808 physPaddleTop 58809 physPaddleTop ...
15636,"(488017, 2, physPaddleTop)",58810 physPaddleTop 58811 physPaddleTop ...
15637,"(488127, 1, physPaddleTop)",58816 physPaddleTop 58817 physPaddleTop ...
